In [1]:
#-----------------------------------
#--        Catherine Baugher      --
#--       Coverage Algorithm      --
#-----------------------------------
#!/usr/bin/env python
import csv
import operator
import pprint
import sys
from copy import copy

#functions
#GREEDY SELECTION FNS BELOW
#------------------------------
#evalResults: given a dictionary of seqs mapped to predictions
#             determine results of the evaluation (FN, TN, FP, TP, error, accuracy etc)
def evalResults(dictofseqs,givendat,i):
    falsepos = falseneg = truepos = trueneg = 0
    for x in range (1,len(givendat)):
        if(givendat[x][0] in dictofseqs):
            if(dictofseqs[givendat[x][0]] != givendat[x][len(givendat[0])-1]):
                if(dictofseqs[givendat[x][0]] == "1.0"):
                    falsepos += 1
                else:
                    falseneg += 1
            else:
                if(dictofseqs[givendat[x][0]] == "1.0"):
                    truepos += 1
                else:
                    trueneg += 1
    tot = len(dictofseqs)
    overallerror = (falseneg+falsepos)/(trueneg+falseneg+falsepos+truepos)
    sens = truepos/(truepos+falseneg)
    spec = trueneg/(trueneg+falsepos)
    k = str(i)
    print("[" + k + "]" + "False Positives: " + str(falsepos))
    print("[" + k + "]" + "False Negatives: " + str(falseneg))
    print("[" + k + "]" + "True Positives: " + str(truepos))
    print("[" + k + "]" + "True Negatives: " + str(trueneg))
    print("[" + k + "]" + "Proportion of false positives: " + str((falsepos/tot)))
    print("[" + k + "]" + "Proportion of false negatives:" + str((falseneg/tot)))
    print("[" + k + "]" + "Accuracy: " + str((1-overallerror)))
    print("[" + k + "]" + "Error rate: " + str(overallerror))
    print("[" + k + "]" + "Sensitivity: " + str(sens))
    print("[" + k + "]" + "Specificity: " + str(spec))

def runGreedyOnTest(model, testdata):
    numrows = len(testdata)
    numcols = len(testdata[0])-1
    results = {}
    for x in range(1, numrows):
        for y in range(1, numcols): #program won't be able to see background/foreground
            if (testdata[x][y] == "1.0"):
                if (testdata[0][y] in model):
                    results[testdata[x][0]] = "1.0"

    for x in range(0,numrows):
        if(testdata[x][0] not in results):
            results[testdata[x][0]] = "-1.0"
    return results

In [2]:
#if(len(sys.argv) > 1):
#    datset = sys.argv[1]
#else:
#    datset = input("What is the name of the dataset (looks at ../featurefilter)?: ")
datset = "filtered_occurrenceandfpCOUNTS"
path = "../../featurefilter/" + datset + ".csv"
with open(path,'r') as f:
    it = csv.reader(f)
    listit = list(it)

In [3]:
lenseq = len(listit)
seqnames = []
F = []
infoF = []
SF = []
store = {}
perc = 0
for x in range(1,lenseq):
    for y in range(0, len(listit[0])-1):
        if y == 0:
            seqnames.append(listit[x][y])
            continue
        if listit[x][y] == "1.0" and listit[x][len(listit[0])-1] == "1.0":
            if listit[0][y] not in store:
                store[listit[0][y]] = list()
            store[listit[0][y]].append(listit[x][0])
print("num feats: " + str(len(store)))
store

num feats: 430


{'A2M_GRCh37_12:9246106-9246106_Missense_Mutation_SNP_C_T_T': ['TCGA-AA-3872',
  'TCGA-AA-A00N'],
 'AADACL4_GRCh37_1:12726318-12726318_Missense_Mutation_SNP_G_A_A': ['TCGA-AA-3811',
  'TCGA-AA-A00N'],
 'ABCA8_GRCh37_17:66881414-66881414_Missense_Mutation_SNP_C_T_T': ['TCGA-AA-3977',
  'TCGA-AA-A00N'],
 'ABCA8_GRCh37_17:66899514-66899514_Missense_Mutation_SNP_C_T_T': ['TCGA-AA-3966',
  'TCGA-AA-A00N'],
 'ABCB8_GRCh37_7:150733024-150733024_Missense_Mutation_SNP_G_A_A': ['TCGA-AA-3949',
  'TCGA-AG-A002'],
 'ABCC9_GRCh37_12:22078907-22078907_Silent_SNP_G_A_A': ['TCGA-AA-3977',
  'TCGA-AG-A002'],
 'ABCG1_GRCh37_21:43716283-43716283_Silent_SNP_C_T_T': ['TCGA-AA-A010',
  'TCGA-AG-A002'],
 'ABL1_GRCh37_9:133729569-133729569_Silent_SNP_C_T_T': ['TCGA-AA-A00N',
  'TCGA-AG-A002'],
 'ACACA_GRCh37_17:35518742-35518742_Missense_Mutation_SNP_G_A_A': ['TCGA-AA-A00N',
  'TCGA-AG-A002'],
 'ACTL7B_GRCh37_9:111617150-111617150_Missense_Mutation_SNP_C_T_T': ['TCGA-AA-3994',
  'TCGA-AA-A010'],
 'ACVR2A_GRCh

In [4]:
from pprint import pprint
while(True):
	max = 0
	sel = ""
	count = 0
	for motif in store:
		#store[motif] = [x for x in store[motif] if x != '']
		if len(store[motif]) > max:
			max = len(store[motif])
			sel = motif
	if sel == "":
		break
	#calculate new percent
	for x in range(1,lenseq):
		if(listit[x][0] in store[sel]):
			if(listit[x][len(listit[0])-1] == "1.0"): #positive
				count += 1
	perc += float(count/65)
	myTup = (sel,perc)
	infoF.append(myTup)
	F.append(sel)
	for seq in store[sel]:
		SF.append(seq)
		
	#print selected sequences (SF)
	#print(store[sel])
	newguy = {} #sequentially copy the new values into here
	for delseqs in store:
		if delseqs not in newguy:
			newguy[delseqs] = list()
			newguy[delseqs] = [y if y not in store[sel] else None for y in store[delseqs]]
			newguy[delseqs] = list(filter((None).__ne__, newguy[delseqs]))
	store = newguy
	store.pop(sel, None)


foreback = {}
#get pos/neg of samples for data purposes
for x in range(1,lenseq):
	if(listit[x][0] in SF):
		foreback[listit[x][0]] = listit[x][len(listit[0])-1]

#F now contains the selected features
print(infoF)
print("\nnum feats: " + str(len(infoF)))
print("-----------------------------------")
print(foreback)


evalResults(runGreedyOnTest(F,listit),listit,0)

[('GRB14_GRCh37_2:165365288-165365296_In_Frame_Del_DEL_TTTTTTTTT_-_-', 0.12307692307692308), ('PRDM2_GRCh37_1:14108749-14108757_In_Frame_Del_DEL_AAAAAAAAA_-_-', 0.16923076923076924), ('CTNND2_GRCh37_5:11117571-11117571_Silent_SNP_G_A_A', 0.2153846153846154), ('TP53_GRCh37_17:7577120-7577120_Missense_Mutation_SNP_C_T_T', 0.26153846153846155), ('MSH6_GRCh37_2:48030640-48030647_Frame_Shift_Del_DEL_CCCCCCCC_-_-', 0.2923076923076923), ('FBXW7_GRCh37_4:153247289-153247289_Missense_Mutation_SNP_G_A_A', 0.3230769230769231), ('BLM_GRCh37_15:91304139-91304147_In_Frame_Del_DEL_AAAAAAAAA_-_-', 0.35384615384615387), ('DNMT1_GRCh37_19:10246814-10246814_Missense_Mutation_SNP_C_C_G', 0.38461538461538464), ('TUBA3C_GRCh37_13:19752431-19752431_Silent_SNP_G_A_A', 0.4153846153846154), ('OR52L1_GRCh37_11:6008035-6008035_Silent_SNP_C_T_T', 0.4461538461538462), ('CADPS_GRCh37_3:62464048-62464048_Missense_Mutation_SNP_C_T_T', 0.47692307692307695), ('SLC30A8_GRCh37_8:118183325-118183325_Silent_SNP_C_T_T', 0.50